In [1]:
import pandas as pd
import numpy as np
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from datetime import datetime
import pandas as pd
import json
import os

In [2]:
csv_folder_path = '/Users/enrique/Documents/03_LeWagon/Final_Project/From_Fadri'

all_articles_list = []

for filename in os.listdir(csv_folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(csv_folder_path, filename)
        try:
            df = pd.read_csv(file_path)
            # Ensure the required columns are present
            required_cols = ['title', 'description', 'content', 'url', 'image', 'publishedAt', 'source']
            if all(col in df.columns for col in required_cols):
                all_articles_list.append(df[required_cols])
            else:
                print(f"Skipping {filename}: Missing required columns.")
        except Exception as e:
            print(f"Error reading {filename}: {e}")

if all_articles_list:
    combined_df = pd.concat(all_articles_list, ignore_index=True)
else:
    combined_df = pd.DataFrame(columns=['title', 'description', 'content', 'url', 'image', 'publishedAt', 'source'])
    print("No valid CSV files found with required columns.")


output_csv_path = '/Users/enrique/Documents/03_LeWagon/Final_Project/Articles_Combined/combined_articles.csv'
combined_df.to_csv(output_csv_path, index=False)

print(f"Combined data saved to {output_csv_path}")

Combined data saved to /Users/enrique/Documents/03_LeWagon/Final_Project/Articles_Combined/combined_articles.csv


# Dataset Overview

In [3]:
df = pd.read_csv("/Users/enrique/Documents/03_LeWagon/Final_Project/Articles_Combined/combined_articles.csv")
df.head()

,title,description,content,url,image,publishedAt,source
0,Corre raises €4m from sale of Dutch battery en...,Shares in renewable energy storage developer h...,"Corre Energy, a developer of storage systems f...",https://www.irishtimes.com/business/2024/12/19...,https://www.irishtimes.com/resizer/v2/TIJLAB7T...,2024-12-19T11:42:00Z,"{'name': 'The Irish Times', 'url': 'https://ww..."
1,Giant Underground 'Batteries' Are Shaping the ...,We're wasting too much of the clean energy we ...,Solar panels and wind turbines give the world ...,https://gizmodo.com/giant-underground-batterie...,https://gizmodo.com/app/uploads/2024/12/hydros...,2024-12-07T17:00:48Z,"{'name': 'Gizmodo', 'url': 'https://gizmodo.com'}"
2,Dudley Council refusal of Halesowen renewable ...,Halesowen renewable energy storage facility ap...,Our community members are treated to special o...,https://www.birminghammail.co.uk/black-country...,https://i2-prod.birminghammail.co.uk/incoming/...,2024-11-24T05:39:00Z,"{'name': 'Birmingham Live', 'url': 'https://ww..."
3,India's Power Ministry Empowers Renewable Stor...,The Union power ministry urges states to waive...,The Union power ministry of India has directed...,https://www.devdiscourse.com/article/business/...,https://www.devdiscourse.com/remote.axd?https:...,2024-11-20T11:20:31Z,"{'name': 'Devdiscourse', 'url': 'https://www.d..."
4,India and Australia Unveil Ambitious Renewable...,India and Australia have launched the Renewabl...,India and Australia have inaugurated a Renewab...,https://www.devdiscourse.com/article/headlines...,https://www.devdiscourse.com/remote.axd?https:...,2024-11-19T18:42:34Z,"{'name': 'Devdiscourse', 'url': 'https://www.d..."


In [4]:
df.shape

(72, 7)

### Date column needs to be **dtype** format!

In [5]:
df['Date'] = pd.to_datetime(df['publishedAt'], dayfirst=True, errors='coerce')
print(df['Date'].dtype)

datetime64[ns, UTC]


/var/folders/52/7ccrs33x6bl334rw1cb76c0h0000gn/T/ipykernel_76353/1960375795.py:1: UserWarning: Parsing dates in %Y-%m-%dT%H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['publishedAt'], dayfirst=True, errors='coerce')


In [6]:
#unique_domains = df['source'].unique()
#unique_domains

In [7]:
df.head(3)

,title,description,content,url,image,publishedAt,source,Date
0,Corre raises €4m from sale of Dutch battery en...,Shares in renewable energy storage developer h...,"Corre Energy, a developer of storage systems f...",https://www.irishtimes.com/business/2024/12/19...,https://www.irishtimes.com/resizer/v2/TIJLAB7T...,2024-12-19T11:42:00Z,"{'name': 'The Irish Times', 'url': 'https://ww...",2024-12-19 11:42:00+00:00
1,Giant Underground 'Batteries' Are Shaping the ...,We're wasting too much of the clean energy we ...,Solar panels and wind turbines give the world ...,https://gizmodo.com/giant-underground-batterie...,https://gizmodo.com/app/uploads/2024/12/hydros...,2024-12-07T17:00:48Z,"{'name': 'Gizmodo', 'url': 'https://gizmodo.com'}",2024-12-07 17:00:48+00:00
2,Dudley Council refusal of Halesowen renewable ...,Halesowen renewable energy storage facility ap...,Our community members are treated to special o...,https://www.birminghammail.co.uk/black-country...,https://i2-prod.birminghammail.co.uk/incoming/...,2024-11-24T05:39:00Z,"{'name': 'Birmingham Live', 'url': 'https://ww...",2024-11-24 05:39:00+00:00


In [8]:
df.drop(columns=['image', 'publishedAt'])
df.head(2)

,title,description,content,url,image,publishedAt,source,Date
0,Corre raises €4m from sale of Dutch battery en...,Shares in renewable energy storage developer h...,"Corre Energy, a developer of storage systems f...",https://www.irishtimes.com/business/2024/12/19...,https://www.irishtimes.com/resizer/v2/TIJLAB7T...,2024-12-19T11:42:00Z,"{'name': 'The Irish Times', 'url': 'https://ww...",2024-12-19 11:42:00+00:00
1,Giant Underground 'Batteries' Are Shaping the ...,We're wasting too much of the clean energy we ...,Solar panels and wind turbines give the world ...,https://gizmodo.com/giant-underground-batterie...,https://gizmodo.com/app/uploads/2024/12/hydros...,2024-12-07T17:00:48Z,"{'name': 'Gizmodo', 'url': 'https://gizmodo.com'}",2024-12-07 17:00:48+00:00


# Text Cleaning:
**Preprocessing:** lowercase, delete numbers, punctuation and symbols (#"*!&%), splitting, tokenizing?, removing stopwords, lemmatizing

In [21]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    #text = text.translate(str.maketrans("", "", string.punctuation))#takes 2 arguments "", "", for string.punctuation
    text = re.sub(r".*'name':\s*'([^']+)'.*", r'\1', text)
    text = re.sub(r'^name\s+(.+?)\s+url\s+https.*', r'\1', text)
    #text = ''.join(char for char in text if not char.isdigit())

    tokens = text.split()
    #tokens = [word for word in tokens if word not in stop_words]
    #tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

df['Clean_Title'] = df['title'].apply(preprocess_text)
df['Clean_Description'] = df['description'].apply(preprocess_text)
df['Clean_Source'] = df['source'].apply(preprocess_text)
df['Clean_Content'] = df['content'].apply(preprocess_text)
df['Clean_Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

df_clean = df.copy()

df_clean = df.drop(columns=['url', 'image', 'publishedAt', 'title', 'description', 'content', 'source', "Date"])
df_clean.head()

,Clean_Title,Clean_Description,Clean_Source,Clean_Content,Clean_Date
0,corre raises €4m from sale of dutch battery en...,shares in renewable energy storage developer h...,the irish times,"corre energy, a developer of storage systems f...",2024-12-19
1,giant underground 'batteries' are shaping the ...,we're wasting too much of the clean energy we ...,gizmodo,solar panels and wind turbines give the world ...,2024-12-07
2,dudley council refusal of halesowen renewable ...,halesowen renewable energy storage facility ap...,birmingham live,our community members are treated to special o...,2024-11-24
3,india's power ministry empowers renewable stor...,the union power ministry urges states to waive...,devdiscourse,the union power ministry of india has directed...,2024-11-20
4,india and australia unveil ambitious renewable...,india and australia have launched the renewabl...,devdiscourse,india and australia have inaugurated a renewab...,2024-11-19


In [22]:
df_clean.Clean_Source.value_counts().sort_values(ascending=False)

Clean_Source
jimmys post                        12
devdiscourse                        9
livemint                            6
the irish times                     5
interesting engineering             5
the indian express                  3
seeking alpha                       2
the conversation uk                 2
slashgear                           2
techcrunch                          2
the australian financial review     2
reuters                             2
gizmodo                             1
birmingham live                     1
the boston globe                    1
the verge                           1
the financial express               1
cbc.ca                              1
elliotlaketoday.com                 1
guelphtoday                         1
bradfordtoday                       1
abc news                            1
baytoday                            1
vancouver is awesome                1
sootoday                            1
spark chronicles                    1

# Sentiment Analysis

In [23]:
#%pip install transformers torch
#%pip install datasets
#%pip install 'transformers[torch]'
# %pip install vaderSentiment


In [24]:
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from collections import Counter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset, ClassLabel
from transformers import TrainingArguments
from transformers import Trainer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


[nltk_data] Downloading package punkt to /Users/enrique/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Setting up different models to test their performance

## Comparing different models: RobErta, distilBERT, VADER

In [26]:
#sentiment_pipeline_roberta = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", return_all_scores=True)
sentiment_pipeline_distilbert = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", return_all_scores=True)
#sentiment_pipeline_vader = SentimentIntensityAnalyzer()

# Adding fine-tune models into the equation
"""sentiment_pipeline_distilbert_V3 = pipeline(
    "sentiment-analysis",
    model="/Users/enrique/Documents/03_LeWagon/Final_Project/FineTuned-models/finetuned_distilbert_V3",
    return_all_scores=True)
"""

Device set to use mps:0


'sentiment_pipeline_distilbert_V3 = pipeline(\n    "sentiment-analysis",\n    model="/Users/enrique/Documents/03_LeWagon/Final_Project/FineTuned-models/finetuned_distilbert_V3",\n    return_all_scores=True)\n'

## We need to devide full articles into ***chunks*** of a **N** of sentences.

In [27]:
def split_into_chunks(text, max_sentences=5):
    "Devide a text in chunks of N sentences"
    sentences = sent_tokenize(text)
    return [' '.join(sentences[i:i+max_sentences]) for i in range(0, len(sentences), max_sentences)]


In [31]:
def analyze_sentiment_chunked(text, sentiment_pipeline):
    "Analyse sentiment of chunks and labels 'mixed' if there is a meaningful draw"

    if hasattr(sentiment_pipeline, 'polarity_scores'):
        return pd.Series(sentiment_pipeline.polarity_scores(text[:514]))

    else:
        return pd.Series(sentiment_pipeline(text[:514]))#, truncation=True)[0] for chunk in chunks]


In [ ]:
#test = df_sample['Clean_Content'][0]
#test
#chunks = split_into_chunks(test)
#len(chunks)

In [ ]:
#[sentiment_pipeline_distilbert_conf(chunk, truncation=True) for chunk in chunks]

In [ ]:
#[sentiment_pipeline_distilbert(chunk, truncation=True) for chunk in chunks]

In [32]:
models = {
    #'roberta': sentiment_pipeline_roberta,
    'distilbert': sentiment_pipeline_distilbert,
    #'vader': sentiment_pipeline_vader,
    #'distilbert_V3': sentiment_pipeline_distilbert_V3
    }
models

{'distilbert': <transformers.pipelines.text_classification.TextClassificationPipeline at 0x165eee350>}

In [48]:
df_sample = df_clean.copy()
df_sample.head(2)

,Clean_Title,Clean_Description,Clean_Source,Clean_Content,Clean_Date
0,corre raises €4m from sale of dutch battery en...,shares in renewable energy storage developer h...,the irish times,"corre energy, a developer of storage systems f...",2024-12-19
1,giant underground 'batteries' are shaping the ...,we're wasting too much of the clean energy we ...,gizmodo,solar panels and wind turbines give the world ...,2024-12-07


In [49]:
for key, value in models.items():
    print(f"Processing with model: {key}")
    preds = df_sample['Clean_Content'].apply(lambda x: analyze_sentiment_chunked(x, value))
    preds.columns = [f"{key}_{col}" for col in preds.columns]
    df_sample = pd.concat([df_sample, preds], axis=1)
    print(f"Processing with model: {key} is done ✅")
#df_sample.iloc[0]['Clean Article Text']

Processing with model: distilbert
Processing with model: distilbert is done ✅


In [50]:
df_sample

,Clean_Title,Clean_Description,Clean_Source,Clean_Content,Clean_Date,distilbert_0
0,corre raises €4m from sale of dutch battery en...,shares in renewable energy storage developer h...,the irish times,"corre energy, a developer of storage systems f...",2024-12-19,"[{'label': 'NEGATIVE', 'score': 0.768901526927..."
1,giant underground 'batteries' are shaping the ...,we're wasting too much of the clean energy we ...,gizmodo,solar panels and wind turbines give the world ...,2024-12-07,"[{'label': 'NEGATIVE', 'score': 0.990027010440..."
2,dudley council refusal of halesowen renewable ...,halesowen renewable energy storage facility ap...,birmingham live,our community members are treated to special o...,2024-11-24,"[{'label': 'NEGATIVE', 'score': 0.688948631286..."
3,india's power ministry empowers renewable stor...,the union power ministry urges states to waive...,devdiscourse,the union power ministry of india has directed...,2024-11-20,"[{'label': 'NEGATIVE', 'score': 0.015595884062..."
4,india and australia unveil ambitious renewable...,india and australia have launched the renewabl...,devdiscourse,india and australia have inaugurated a renewab...,2024-11-19,"[{'label': 'NEGATIVE', 'score': 0.000571808195..."
...,...,...,...,...,...,...
67,g-20 meetings: improving energy storage key to...,while delivering the keynote address at the fi...,the indian express,improving energy storage and ensuring energy s...,2023-02-05,"[{'label': 'NEGATIVE', 'score': 0.004108163062..."
68,budget 2023 expectations: making a case for na...,the union 2022-23 budget showed hints of promi...,firstpost,as the government heads into the last full bud...,2023-02-01,"[{'label': 'NEGATIVE', 'score': 0.991499483585..."
69,this innovative new technology turns abandoned...,a proposed plan could utilize an unusual metho...,slashgear,this innovative new technology turns abandoned...,2023-01-22,"[{'label': 'NEGATIVE', 'score': 0.002188169164..."
70,jsw energy secures 500mw battery energy storag...,jsw energy has set an ambitious target of a 50...,livemint,new delhi: jsw energy on thursday said its ste...,2023-01-18,"[{'label': 'NEGATIVE', 'score': 0.925697445869..."


In [51]:
df_sample.distilbert_0[0]

[{'label': 'NEGATIVE', 'score': 0.768901526927948},
 {'label': 'POSITIVE', 'score': 0.2310985028743744}]

In [52]:
df_sample['distilbert_pos_score'] = df_sample.distilbert_0.map(lambda x : x[1]['score'])

In [53]:
df_sample['distilbert_neg_score'] = df_sample.distilbert_0.map(lambda x : x[0]['score'])

In [56]:
df_sample = df_sample.drop(columns=['distilbert_0'])

In [57]:
df_sample.head(3)

,Clean_Title,Clean_Description,Clean_Source,Clean_Content,Clean_Date,distilbert_pos_score,distilbert_neg_score
0,corre raises €4m from sale of dutch battery en...,shares in renewable energy storage developer h...,the irish times,"corre energy, a developer of storage systems f...",2024-12-19,0.231099,0.768902
1,giant underground 'batteries' are shaping the ...,we're wasting too much of the clean energy we ...,gizmodo,solar panels and wind turbines give the world ...,2024-12-07,0.009973,0.990027
2,dudley council refusal of halesowen renewable ...,halesowen renewable energy storage facility ap...,birmingham live,our community members are treated to special o...,2024-11-24,0.311051,0.688949


In [58]:
export_path = '/Users/enrique/Documents/03_LeWagon/Final_Project/Articles_Combined/combined_articles_with_sentiment.csv'
df_sample.to_csv(export_path, index=False)
print(f"Data with sentiment analysis saved to {export_path}")

Data with sentiment analysis saved to /Users/enrique/Documents/03_LeWagon/Final_Project/Articles_Combined/combined_articles_with_sentiment.csv


In [43]:
"""for key, value in models.items():
    print(f"Processing with model: {key}")
    df_sample[[f'Sentiment_{key}', f'Sentiment_Conf_{key}']] = df_sample['Clean_Content'].apply(lambda x: analyze_sentiment_chunked(x, value))
    print(f"Processing with model: {key} is done ✅")
#df_sample.iloc[0]['Clean Article Text']
"""

'for key, value in models.items():\n    print(f"Processing with model: {key}")\n    df_sample[[f\'Sentiment_{key}\', f\'Sentiment_Conf_{key}\']] = df_sample[\'Clean_Content\'].apply(lambda x: analyze_sentiment_chunked(x, value))\n    print(f"Processing with model: {key} is done ✅")\n#df_sample.iloc[0][\'Clean Article Text\']\n'

In [44]:
"""
print(df_sample['Sentiment_roberta'].value_counts(),'\n')
print(df_sample['Sentiment_distilbert'].value_counts(),'\n')
print(df_sample['Sentiment_vader'].value_counts(),'\n')
print(df_sample['Sentiment_distilbert_complete_V3'].value_counts(),'\n')
print(df_sample['Sentiment_distilbert_complete'].value_counts(),'\n')
"""

"\nprint(df_sample['Sentiment_roberta'].value_counts(),'\n')\nprint(df_sample['Sentiment_distilbert'].value_counts(),'\n')\nprint(df_sample['Sentiment_vader'].value_counts(),'\n')\nprint(df_sample['Sentiment_distilbert_complete_V3'].value_counts(),'\n')\nprint(df_sample['Sentiment_distilbert_complete'].value_counts(),'\n')\n"

In [ ]:
"""
print(df_sample[df_sample['Sentiment_roberta'] == 'neutral'].sample()['Clean_Content'].values, '\n')
#print(df_sample[df_sample['Sentiment_distilbert'] == 'NEUTRAL'].sample()['Clean_Content'].values, '\n')
print(df_sample[df_sample['Sentiment_vader'] == 'NEUTRAL'].sample()['Clean_Content'].values,'\n')
print(df_sample[df_sample['Sentiment_distilbert_complete_V3'] == 'NEUTRAL'].sample()['Clean_Content'].values, '\n')
print(df_sample[df_sample['Sentiment_distilbert_complete'] == 'neutral'].sample()['Clean_Content'].values, '\n')
print(df_sample[df_sample['Sentiment_distilbert_weighted'] == 'NEUTRAL'].sample()['Clean_Content'].values, '\n')


sentiment_pipeline = pipeline("sentiment-analysis", model= "cardiffnlp/twitter-roberta-base-sentiment-latest", device=0)
df_sample = df_clean.copy()

#"cardiffnlp/twitter-roberta-base-sentiment-latest"


analyzer = SentimentIntensityAnalyzer()
text = "hotter summers, drier forests, rising waters: climate change is not just a threat to our future, its hurting our world right now. while there are many ways human activity has brought about climate change, global electricity generation sources are among the leading culprits. despite small upticks in the supply of wind and solar power, we have not yet reached a point where we are able to dislodge the fossil fuels that are entrenched in the power mix of many countries. but why is this still the case? since renewable sources deliver an intermittent supply of power, we also need a way to store this energy to meet the demand of the grid when the sun is not shining, or the wind is not blowing. this is a major challenge, as the switch to renewable power also requires establishing long lasting, safe and affordable energy storage systems. as such, finding a cheap, safe and alternative battery to lithium is the key to moving the needle to a completely renewable power sector. beyond lithium-ion batteries as with electric vehicles, lithium-ion batteries have become a popular option for the grid, as they offer a high energy density, modular solution for energy storage. but the use of lithium-ion batteries has also brought along its own challenges with high cost of materials, risk of fire and explosion and lack of recycling practices limiting the widespread adoption of lithium-ion batteries for the grid. the canadian press/dave chidley one incredibly promising option to replace lithium for grid scale energy storage is the rechargeable zinc-ion battery. emerging only within the last 10 years, zinc-ion batteries offer many advantages over lithium. these include cheaper material costs, increased safety and easier recycling options. read more: batteries are the environmental achilles heel of electric vehicles – unless we repair, reuse and recycle them with grid-scale energy storage potential at a considerably cheaper cost — and higher levels of safety — widespread commercialization of zinc-ion batteries could be exactly what is needed to integrate renewables into energy infrastructure in canada and other countries. the cost of a battery for canada to reach the decarbonization targets set in the canadian net-zero emissions accountability act, including a grid powered by 90 per cent renewable electricity, the deployment of zinc-ion batteries will be crucial. studies have shown that for renewables to become the source of 90 to 95 per cent of all electricity, the cost of energy storage must be below us$150/kwh. modern lithium-ion systems are still sitting around us$350/kwh. in part, this is due to high manufacturing costs and their reliance on expensive raw materials to achieve the high energy density needed for modern electric vehicles. the canadian press/john woods zinc-ion batteries on the other hand, could solve the cost and abundance issues. using inexpensive, abundant materials such as zinc and manganese not only makes them cheaper to produce, but lowers risk from supply chain disruptions or material shortages that affect lithium-ion materials such as lithium and cobalt. the annual production of zinc globally is over 100 times that of lithium. not to mention that demand for lithium and cobalt is anticipated to outweigh the supply within the next decade. zinc is a safer option with rigorous safety standards being created for batteries used in homes, factories or within the electrical grid, safety is key to getting the public to embrace them. in this way, zinc-ion batteries offer further advantage. the flammable and toxic solvent based electrolyte of lithium-ion batteries is replaced with a water-based alternative, removing the risk of fire and explosion. read more: we could need 6 times more of the minerals used for renewables and batteries. how can we avoid a huge increase in mining impacts? conversely, the safe disposal of lithium-ion batteries can also be a difficult task, as they contain toxic compounds. recycling these batteries is currently economically infeasible due to high costs leading to large numbers of spent cells ending up in landfills. fortunately, zinc-ion batteries simplify end of life treatment. the nontoxic, aqueous electrolyte used in zinc-ion batteries means that well established methods like those for lead-acid battery disposal can be used. also, the metallic zinc anode could be easily reused in new batteries. the future of energy storage to reach its goal of 90 per cent renewable energy by 2030, canada must look for alternatives to lithium-ion batteries to enable decarbonization of its power sector. leveraging the cost, abundance and safety benefits of zinc-ion batteries, canada can accelerate the integration of wind and solar power across the nation. zinc-ion batteries support canada’s decarbonization goals and prove an opportunity to capitalize on a rapidly expanding battery market. while zinc-ion batteries are a relatively new technology, their potential to support grid scale energy storage within canada and worldwide cannot be understated. with the help of canadian research and manufacturing, including efforts from mcmaster university and dartmouth, n.s.-based salient energy inc., the integration of zinc-ion batteries could become a reality within the next several years, establishing canada as an industry leader."
scores = analyzer.polarity_scores(text)
scores
# Returns: {'neg': 0.0, 'neu': 0.6, 'pos': 0.4, 'compound': 0.7}
"""

"\nprint(df_sample[df_sample['Sentiment_roberta'] == 'neutral'].sample()['Clean_Content'].values, '\n')\n#print(df_sample[df_sample['Sentiment_distilbert'] == 'NEUTRAL'].sample()['Clean_Content'].values, '\n')\nprint(df_sample[df_sample['Sentiment_vader'] == 'NEUTRAL'].sample()['Clean_Content'].values,'\n')\nprint(df_sample[df_sample['Sentiment_distilbert_complete_V3'] == 'NEUTRAL'].sample()['Clean_Content'].values, '\n')\nprint(df_sample[df_sample['Sentiment_distilbert_complete'] == 'neutral'].sample()['Clean_Content'].values, '\n')\nprint(df_sample[df_sample['Sentiment_distilbert_weighted'] == 'NEUTRAL'].sample()['Clean_Content'].values, '\n')\n"